In [ ]:
# -------------------------
# BASE DE DATOS TEMPORAL
# -------------------------

usuarios = []      # Lista de diccionarios con datos del usuario
reservas = []      # Lista de reservas hechas

# Creación de sala de 11 columnas x 10 filas (121 sillas)
# 'O' = disponible | 'X' = ocupado

sala = [
    ['O'] * 11 for _ in range(11)
]

# Películas de ejemplo (el profe dará la definitiva)
funciones = [
    {"dia": "Sábado", "hora": "3:00 PM", "nombre": "Interestelar"},
    {"dia": "Sábado", "hora": "6:00 PM", "nombre": "Interstate 99"},
    {"dia": "Domingo", "hora": "2:00 PM", "nombre": "Cazafantasmas"}
]
# from datos import usuarios # This line is causing the ModuleNotFoundError. Remove or comment it out.

# -------------------------------------
# VALIDACIONES DE ENTRADA DEL USUARIO
# -------------------------------------

def validar_nombre(texto):
    """Valida que el nombre o apellido tenga solo letras y mínimo 3 caracteres."""
    if len(texto) < 3:
        return False
    if not texto.isalpha():
        return False
    return True


def validar_documento(doc):
    """Valida que el documento tenga solo números y entre 3 y 15 dígitos."""
    return doc.isdigit() and 3 <= len(doc) <= 15


# -------------------------------------
# REGISTRO DE USUARIOS
# -------------------------------------

def registrar_usuario():
    print("\n=== REGISTRO DE USUARIO ===")

    # NOMBRE
    nombre = input("Ingrese el nombre: ")
    while not validar_nombre(nombre):
        print("❌ Nombre inválido. Debe tener al menos 3 letras y no contener números.")
        nombre = input("Ingrese el nombre: ")

    # APELLIDO
    apellido = input("Ingrese el apellido: ")
    while not validar_nombre(apellido):
        print("❌ Apellido inválido. Debe tener al menos 3 letras y no contener números.")
        apellido = input("Ingrese el apellido: ")

    # DOCUMENTO
    documento = input("Ingrese el documento: ")
    while not validar_documento(documento):
        print("❌ El documento debe ser numérico y contener entre 3 y 15 dígitos.")
        documento = input("Ingrese el documento: ")

    # TIPO DE VÍNCULO
    print("\nTipos de usuario:")
    print("1. Estudiante ($7.500)")
    print("2. Docente ($10.000)")
    print("3. Administrativo ($8.500)")
    print("4. Oficial Interno ($7.000)")
    print("5. Público Externo ($15.000)")

    tipos = ["Estudiante", "Docente", "Administrativo", "Oficial Interno", "Público Externo"]
    precios = [7500, 10000, 8500, 7000, 15000]

    tipo_op = input("Seleccione el tipo (1-5): ")
    while not tipo_op.isdigit() or int(tipo_op) not in range(1, 6):
        print("❌", end=" ")
        tipo_op = input("Seleccione el tipo (1-5): ")

# ----------------------------------
# FUNCIÓN PARA MOSTRAR LA SALA
# ----------------------------------

def mostrar_sala():
    print("\nCINE, CÁMARAS Y UDEA  (O = Disponible / X = Ocupado)\n")

    columnas = "   " + "  ".join([chr(65 + i) for i in range(len(sala[0]))])
    print(columnas)

    for i, fila in enumerate(sala):
        fila_str = f"{chr(65 + i)}  " + "  ".join(fila)
        print(fila_str)


# ----------------------------------
# BUSCAR USUARIO POR DOCUMENTO
# ----------------------------------

def buscar_usuario(documento):
    for u in usuarios:
        if u["documento"] == documento:
            return u
    return None


# ----------------------------------
# REGISTRAR RESERVA
# ----------------------------------

def registrar_reserva():
    print("\n=== REGISTRAR RESERVA ===")

    documento = input("Ingrese el documento del usuario: ")

    usuario = buscar_usuario(documento)
    if usuario is None:
        print("❌ El usuario no está registrado. Primero debe registrarse.")
        return

    mostrar_sala()

    asiento = input("\nSeleccione el asiento (Ej: A5): ").upper()

    if len(asiento) < 2:
        print("❌ Formato inválido. Debe ser como A5.")
        return

    fila_letra = asiento[0]
    col_num = asiento[1:]

    if not col_num.isdigit():
        print("❌ El número de silla debe ser numérico.")
        return

    fila = ord(fila_letra) - 65
    col = int(col_num)

    if fila < 0 or fila >= len(sala) or col < 0 or col >= len(sala[0]):
        print("❌ El asiento no existe.")
        return

    if sala[fila][col] == "X":
        print("❌ El asiento ya está ocupado.")
        return

    sala[fila][col] = "X"

    reservas.append({
        "usuario": usuario,
        "fila": fila_letra,
        "columna": col,
        "precio": usuario["precio"]
    })

    print("\n✔ Reserva realizada con éxito.")
    print(f"Asiento reservado: {fila_letra}{col}")
    print(f"Total a pagar: ${usuario['precio']}\n")


# ----------------------------------
# CANCELAR RESERVA
# ----------------------------------

def cancelar_reserva():
    print("\n=== CANCELAR RESERVA ===")

    documento = input("Ingrese el documento del usuario: ")

    usuario = buscar_usuario(documento)
    if usuario is None:
        print("❌ No existe un usuario con ese documento.")
        return

    reservas_usuario = [r for r in reservas if r["usuario"]["documento"] == documento]

    if not reservas_usuario:
        print("❌ El usuario no tiene reservas activas.")
        return

    print("\nReservas activas del usuario:")
    for i, r in enumerate(reservas_usuario):
        print(f"{i+1}. Asiento {r['fila']}{r['columna']} - ${r['precio']}")

    opcion = input("\nSeleccione la reserva a cancelar: ")

    if not opcion.isdigit() or not (1 <= int(opcion) <= len(reservas_usuario)):
        print("❌ Opción inválida.")
        return

    reserva = reservas_usuario[int(opcion) - 1]

    fila = ord(reserva["fila"]) - 65
    columna = reserva["columna"]
    sala[fila][columna] = "O"

    reservas.remove(reserva)

    print("\n✔ Reserva cancelada exitosamente.")
    print(f"Asiento liberado: {reserva['fila']}{reserva['columna']}")
    # from datos import funciones # Removed this line as funciones is already defined globally

# ----------------------------------
# CONSULTAR FUNCIONES DEL FIN DE SEMANA
# ----------------------------------

def consultar_funciones():
    print("\n========== FUNCIONES DEL FIN DE SEMANA ==========")
    print("============= CINE, CÁMARAS Y UDEA ==============\n")

    if not funciones:
        print("No hay funciones registradas por el momento.")
        return

    # Mostrar funciones en orden
    for f in funciones:
        print(f"📅 {f['dia']}  |  🕒 {f['hora']}  |  🎬 {f['nombre']}")

    print("\nRecuerde que las reservas se realizan desde el menú principal.")
    # from datos import usuarios, reservas # Removed this line as usuarios and reservas are already defined globally

# --------------------------------------
# MENÚ DE ADMINISTRACIÓN
# --------------------------------------

def menu_admin():
    print("\n=== ACCESO A ADMINISTRACIÓN DEL CINE, CÁMARAS Y UDEA ===")

    # Usuario y contraseña predefinidos
    usuario_admin = "admin"
    password_admin = "1234"

    usuario = input("Ingrese usuario administrador: ")
    contraseña = input("Ingrese contraseña: ")

    if usuario != usuario_admin or contraseña != password_admin:
        print("❌ Acceso denegado. Usuario o contraseña incorrectos.")
        return

    while True:
        print("\n======= PANEL ADMINISTRATIVO =======")
        print("1. Total de reservas registradas")
        print("2. Total de tiquetes vendidos")
        print("3. Total pago realizado (recaudo)")
        print("4. Promedio por venta diario del cine")
        print("5. Lista de usuarios")
        print("6. Usuario con mayor cantidad de reservas")
        print("7. Usuario con menor cantidad de reservas")
        print("8. Salir del panel")

        opcion = input("\nSeleccione una opción: ")

        if opcion == "1":
            total_reservas()

        elif opcion == "2":
            total_tiquetes()

        elif opcion == "3":
            total_recaudo()

        elif opcion == "4":
            promedio_diario()

        elif opcion == "5":
            listar_usuarios()

        elif opcion == "6":
            mayor_reservas()

        elif opcion == "7":
            menor_reservas()

        elif opcion == "8":
            print("\nSaliendo del administrador...")
            break

        else:
            print("❌ Opción inválida.")


# --------------------------------------
# REPORTES
# --------------------------------------

def total_reservas():
    print(f"\n📌 Total de reservas registradas: {len(reservas)}")


def total_tiquetes():
    print(f"\n🎟️ Total de tiquetes vendidos: {len(reservas)}")


def total_recaudo():
    total = sum(r["precio"] for r in reservas)
    print(f"\n💰 Total recaudado: ${total}")


def promedio_diario():
    if len(reservas) == 0:
        print("\nNo hay datos suficientes para calcular el promedio.")
        return

    # Consideramos solo sábado y domingo: 2 días
    promedio = sum(r["precio"] for r in reservas) / 2
    print(f"\n📊 Promedio por día del fin de semana: ${promedio:.2f}")


def listar_usuarios():
    if not usuarios:
        print("\nNo hay usuarios registrados.")
        return

    print("\n=== LISTA DE USUARIOS REGISTRADOS ===")
    for u in usuarios:
        print(f"- {u['nombre']} {u['apellido']} ({u['tipo']})  Doc: {u['documento']}")


def mayor_reservas():
    if not reservas:
        print("\nNo hay reservas registradas.")
        return

    contador = {}  # documento : número de reservas

    for r in reservas:
        doc = r["usuario"]["documento"]
        contador[doc] = contador.get(doc, 0) + 1

    # Usuario con mayor reservas
    doc_mayor = max(contador, key=contador.get)
    cant = contador[doc_mayor]

    usuario = next(u for u in usuarios if u["documento"] == doc_mayor)

    print(f"\n🏆 Usuario con más reservas: {usuario['nombre']} {usuario['apellido']} ({cant} reservas)")


def menor_reservas():
    if not reservas:
        print("\nNo hay reservas registradas.")
        return

    contador = {}

    for r in reservas:
        doc = r["usuario"]["documento"]
        contador[doc] = contador.get(doc, 0) + 1

    doc_menor = min(contador, key=contador.get)
    cant = contador[doc_menor]

    usuario = next(u for u in usuarios if u["documento"] == doc_menor)

    print(f"\n📉 Usuario con menos reservas: {usuario['nombre']} {usuario['apellido']} ({cant} reservas)")
# from .usuarios import registrar_usuario # Removed unnecessary relative import
# from .reservas import registrar_reserva, cancelar_reserva, mostrar_sala # Removed unnecessary relative import
# from .funciones import consultar_funciones # Removed unnecessary relative import
# from .admin import menu_admin # Removed unnecessary relative import

# -------------------------------
# MENÚ PRINCIPAL DEL CINE, CÁMARAS Y UDEA
# -------------------------------

def menu_principal():
    while True:
        print("\n========== CINE, CÁMARAS Y UDEA ==========\n")
        print("1. Registrar Usuario")
        print("2. Registrar Reserva")
        print("3. Cancelar Reserva")
        print("4. Consultar Funciones Fin de Semana")
        print("5. Administrador")
        print("6. Salir")

        opcion = input("\nSeleccione una opción: ")

        if opcion == "1":
            registrar_usuario()

        elif opcion == "2":
            registrar_reserva()

        elif opcion == "3":
            cancelar_reserva()

        elif opcion == "4":
            consultar_funciones()

        elif opcion == "5":
            menu_admin()

        elif opcion == "6":
            print("\nGracias por usar Cine, Cámaras y UdeA. ¡Hasta luego!")
            break

        else:
            print("\n❌ Opción inválida. Intente nuevamente.")


if __name__ == "__main__":
    menu_principal()


========== CINE, CÁMARAS Y UDEA ==========

1. Registrar Usuario
2. Registrar Reserva
3. Cancelar Reserva
4. Consultar Funciones Fin de Semana
5. Administrador
6. Salir

Seleccione una opción: 7

❌ Opción inválida. Intente nuevamente.

========== CINE, CÁMARAS Y UDEA ==========

1. Registrar Usuario
2. Registrar Reserva
3. Cancelar Reserva
4. Consultar Funciones Fin de Semana
5. Administrador
6. Salir

Seleccione una opción: 7

❌ Opción inválida. Intente nuevamente.

========== CINE, CÁMARAS Y UDEA ==========

1. Registrar Usuario
2. Registrar Reserva
3. Cancelar Reserva
4. Consultar Funciones Fin de Semana
5. Administrador
6. Salir

Seleccione una opción: f

❌ Opción inválida. Intente nuevamente.

========== CINE, CÁMARAS Y UDEA ==========

1. Registrar Usuario
2. Registrar Reserva
3. Cancelar Reserva
4. Consultar Funciones Fin de Semana
5. Administrador
6. Salir
